In [34]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import random
from scipy.stats import norm,multivariate_normal

import warnings
warnings.filterwarnings("ignore")
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression



In [35]:
# logging_index
# eval_index

# logging_policy 1
# eval_policy 1
# logging_xshift 1
# eval_xshift 1
# first_component 1
# logging_xy 1
# eval_xy 1
# logging_reward 1
# eval_reward 1
# logging_label 1
# eval_reward 1



In [36]:
# box 1

# xy
# label
# fc

data = pd.read_csv('optdigits.csv',header = None)
from sklearn.utils import shuffle
data = shuffle(data)


label = np.array(data[64]) - 1
xy = np.array(data.drop([64],axis = 1))
n_class = 10
n_dim = xy.shape[1]

train_xy = xy[:int(len(xy)*0.7)]
test_xy = xy[int(len(xy)*0.7):]

train_label = label[:int(len(xy)*0.7)]
test_label = label[int(len(xy)*0.7):]




pca_model = PCA(n_components=1).fit(xy)
fc = pca_model.transform(xy)

train_fc = fc[:int(len(xy)*0.7)]
test_fc = fc[int(len(xy)*0.7):]

train_size = int(len(xy)*0.7)



In [79]:
# box 2

logging_a = 0.8
logging_b = 0.3


def get_distribution(fc,a,b,pca_model=None,test = False):
    if test:
        pdf = np.ones(len(fc))
        pdf = pdf/sum(pdf)
        pdf = pdf.reshape(len(pdf))
        return pdf/sum(pdf), 0, 1
    m = np.min(fc)
    m_bar = np.mean(fc)
    mean = m + (m_bar-m)/a
    std = np.sqrt((m_bar - m)/b)
    pdf = norm(loc = mean,scale = std).pdf(fc)
#     pdf = np.ones(len(fc))
    pdf = pdf/sum(pdf)
    pdf = pdf.reshape(len(pdf))
    return pdf/sum(pdf), mean, std








In [80]:
# # box 3
# # logging policy, eval_policy
# logging_policy_interval = [0.5,0.8]
# eval_policy_interval = [0.8,0.9]

def generate_policy_soften(alpha,beta):
    clf_policy = LogisticRegression(random_state=0).fit(train_xy[:int(train_size*0.1)], train_label[:int(train_size*0.1)])
    predict_label = clf_policy.predict(xy)
    logging_policy = np.zeros((len(xy),n_class))

    p_max = []
    for i in range(len(xy)):
        tmp = np.zeros((1,n_class))
        u = np.random.uniform(-0.5,0.5)
        for t in range(n_class):
            if t == predict_label[i]:
                tmp[0,t] = alpha + u*beta
            else:
                tmp[0,t] = (1-alpha - u*beta)/(n_class-1)
        logging_policy[i] = tmp



    clf_policy = LogisticRegression(random_state=1).fit(train_xy, train_label)
    predict_label = clf_policy.predict(xy)
    eval_policy = np.zeros((len(xy),n_class))
    p_max = []
    for i in range(len(xy)):
        tmp = np.zeros((1,n_class))
#         u = np.random.uniform(-0.05,0.05)
        for t in range(n_class):
            if t == predict_label[i]:
                tmp[0,t] = 0.9
            else:
                tmp[0,t] = (1-0.9)/(n_class-1)
        eval_policy[i] = tmp
    return logging_policy, eval_policy
    

In [81]:
# # box 3
# # logging policy, eval_policy
# logging_policy_interval = [0.5,0.8]
# eval_policy_interval = [0.8,0.9]

def generate_policy_adv(alpha,beta):
    clf_policy = LogisticRegression(random_state=0).fit(xy, label)
    predict_label = clf_policy.predict(xy)
    logging_policy = np.zeros((len(xy),n_class))

    p_max = []
    for i in range(len(xy)):
        tmp = np.zeros((1,n_class))
        u = np.random.uniform(-0.5,0.5)
        for t in range(n_class):
            if t == predict_label[i]:
                tmp[0,t] = 1- alpha - u*beta
            else:
                tmp[0,t] = (alpha + u*beta)/(n_class-1)
        logging_policy[i] = tmp



    clf_policy = LogisticRegression(random_state=1).fit(xy, label)
    predict_label = clf_policy.predict(xy)
    eval_policy = np.zeros((len(xy),n_class))
    p_max = []
    for i in range(len(xy)):
        tmp = np.zeros((1,n_class))
#         u = np.random.uniform(-0.05,0.05)
        for t in range(n_class):
            if t == predict_label[i]:
                tmp[0,t] = 0.9
            else:
                tmp[0,t] = (1-0.9)/(n_class-1)
        eval_policy[i] = tmp
    return logging_policy, eval_policy
    

In [92]:
# sample data
# just sample index and everything can be refered from index.
def take_action(policy):
    action = random.choices([t for t in range(n_class)],policy)[0]
    prob = policy[action]
    return action, prob


def sample_logging_data(xy, label, size, pdf, pdf_eval, logging_policy, eval_policy):
    logging_index = np.zeros(size,np.int)
    logging_action = np.zeros(size,np.int)
    logging_reward = np.zeros(size,np.int)
    index_list = [i for i in range(len(xy))]
    
    for i in range(size):
        t_index = random.choices(index_list,weights=pdf[:train_size])[0]
#         t_index = i
        logging_index[i] = t_index
        
        action,_ = take_action(logging_policy[t_index])
        logging_action[i] = action
        
        logging_reward[i] = int(action == label[t_index])
    
    return logging_index, logging_action, logging_reward




def sample_eval_data(xy, label, size, pdf, pdf_eval, logging_policy, eval_policy):
    eval_index = np.zeros(size,np.int)
    eval_action = np.zeros(size,np.int)
    eval_reward = np.zeros(size,np.int)
    index_list = [i for i in range(len(xy))]
    
    for i in range(size):
        t_index = random.choices(index_list,weights=pdf_eval[train_size:])[0]
        t_index = t_index  + train_size
        eval_index[i] = t_index  
        
        action,_ = take_action(logging_policy[t_index])
        eval_action[i] = action
        
        eval_reward[i] = int(action == label[t_index-train_size])
    
    return eval_index , eval_action, eval_reward



def sample_eval_logging_data(xy, label, size, pdf, pdf_eval, logging_policy, eval_policy):
    eval_index = np.zeros(size,np.int)
    eval_action = np.zeros(size,np.int)
    eval_reward = np.zeros(size,np.int)
    index_list = [i for i in range(len(xy))]
    
    for i in range(size):
        t_index = random.choices(index_list,weights=pdf[train_size:])[0]
        t_index = t_index  + train_size
        eval_index[i] = t_index  
        
        action,_ = take_action(logging_policy[t_index])
        eval_action[i] = action
        
        eval_reward[i] = int(action == label[t_index-train_size])
    
    return eval_index , eval_action, eval_reward




        



In [93]:
def p_xshift(train_x,test_x):
    from sklearn.linear_model import LogisticRegression
    X = np.concatenate((train_x,test_x),axis = 0)
#     X = np.sum(X,axis = 1,keepdims = True)
    Y = np.concatenate((np.ones(len(train_x)),np.zeros(len(test_x))),axis = 0)
    clf = LogisticRegression().fit(X,Y)
    return clf

def loggingpolicy(x,y):
    from sklearn.linear_model import LogisticRegression
    clf = LogisticRegression().fit(x,y)
    return clf

In [121]:
def evaluate_ips(logging_index, logging_action, logging_reward,\
                 eval_logging_index, eval_logging_action, eval_logging_reward,\
                 xy,label,  pdf, pdf_eval, logging_policy, eval_policy,GT):
#     GT = np.mean(eval_reward)
    ips_list = []
    snips_list = []
    ips_r_list = []
    
    ips_list_unknown = []
    snips_list_unknown = []
    ips_r_list_unknown = []
    
    
    
    for k in range(1):
        p_w = my_bound(eval_policy[eval_logging_index, eval_logging_action]/logging_policy[eval_logging_index, eval_logging_action])
        x_w = pdf_eval[eval_logging_index]/pdf[eval_logging_index]

        ips = p_w*eval_logging_reward
        ips_loss = (np.mean(ips) - GT)**2
        ips_list.append(ips_loss)

        snips = p_w*eval_logging_reward/np.mean(p_w)
        snips_loss = (np.mean(snips) - GT)**2
        snips_list.append(snips_loss)

        ips_r = x_w*p_w*eval_logging_reward
        ips_r_loss = (np.mean(ips_r) - GT)**2
        ips_r_list.append(ips_r_loss)




    #     unknown:
#         clf_xshift = p_xshift(xy[logging_index],xy[eval_index])
#         clf_logging = loggingpolicy(xy[logging_index],logging_action)


#         logging_predicted = clf_logging.predict_proba(xy[eval_index])[np.arange(len(eval_index)),np.array(eval_action,np.int)]
#         p_w = eval_policy[eval_reward, eval_action]/logging_predicted

#         xshift_logging = clf_xshift.predict_proba(xy[eval_reward])
#         x_w = xshift_logging[:,1]/xshift_logging[:,0]



#         ips = p_w*eval_reward
#         ips_loss_unknown = (np.mean(ips) - GT)**2
#         ips_list_unknown.append(ips_loss_unknown)

#         snips = p_w*eval_reward/np.mean(p_w)
#         snips_loss_unknown = (np.mean(snips) - GT)**2
#         snips_list_unknown.append(snips_loss_unknown)

#         ips_r = x_w*p_w*eval_reward
#         ips_r_loss_unknown = (np.mean(ips_r) - GT)**2
#         ips_r_list_unknown.append(ips_r_loss_unknown)

    
    return np.mean(ips_list),np.mean(snips_list),np.mean(ips_r_list)
#                 np.mean(ips_list_unknown),np.mean(snips_list_unknown),np.mean(ips_r_list_unknown),
    
    

    
    
    
    
    
    
    
    
    

In [122]:
from ope_v3 import *


class WEIGHT_DATA_SET(data.Dataset):
    '''
    dataset class with instance weight 
    '''
    def __init__(self, data,reward, weights):
        '''
        weights are same dimensional witi original data
        '''
        self.data = torch.tensor(data)
        self.reward = torch.tensor(reward)
        self.weights = torch.tensor(weights)

    def __getitem__(self, index):
        img = self.data[index] 
        target = self.reward[index]
        weight = self.weights[index]
        return img, target, weight

    def __len__(self):
        return len(self.data)

# r is xshift weight
# w is policy shift weight
def train_robust_regression(logging_index, logging_action, logging_reward,\
                 eval_index, eval_action, eval_reward,\
                 xy,label,  pdf, pdf_eval, logging_policy, eval_policy,\
                            dm=False,xshift_known = True, policy_known = True,use_xshift = True):
#     print('train models with true density ratios')
    Myy_robust_list = []
    Myx_robust_list = []
    model_robust_list = []
    
    feature = xy[logging_index]
    eval_xshift = pdf_eval[logging_index]
    logging_xshift = pdf[logging_index]
    eval_policy = eval_policy[logging_index,logging_action]
    logging_policy = logging_policy[logging_index,logging_action]
    
    
    for i in range(n_class):
        ## generate training set for action i

        idx = (logging_action == i)


        data_i = feature[idx]
        reward_i = logging_reward[idx]
        eval_xshift_i = eval_xshift[idx]
        logging_xshift_i = logging_xshift[idx]
        eval_policy_i = eval_policy[idx]
        logging_policy_i = logging_policy[idx]

        train_size = len(data_i)
        if xshift_known:
            r = logging_xshift_i/eval_xshift_i
        if policy_known:
            w = logging_policy_i/eval_policy_i
        if use_xshift:
            w = r * w
        
        if dm:
            w = torch.ones(len(data_i))
        

        weight_st = my_bound(w)
        weighted_train = WEIGHT_DATA_SET(data_i,reward_i, w)

        train_model = Net()

        validate_size = int(0.1*train_size)

        validate_loader = data.DataLoader(data.Subset(weighted_train, range(0, validate_size)),
            batch_size=32, shuffle=True)
        # 10% validation set
        train_loader = data.DataLoader(data.Subset(weighted_train, range(validate_size, train_size)),
            batch_size=32, shuffle=True, )

        train_model, Myy, Myx, _, _, _ = train_validate_test(args,args.lr_robust, 150, "regression", 'cpu', 'False', train_model,
            train_loader, None , validate_loader, n_class , 0.000, d = n_dim,testflag = False)

        model_robust_list.append(train_model)
        Myy_robust_list.append(Myy)
        Myx_robust_list.append(Myx)
    return model_robust_list, Myy_robust_list, Myx_robust_list




In [123]:
def predict_regression(model_robust_list, Myy_robust_list, Myx_robust_list,\
                        logging_action,\
                       xy,  pdf, pdf_eval, logging_policy, eval_policy,\
                       dm = False,xshift_known = True, policy_known = True,use_xshift = True):
    
    
    
    if xshift_known:
        r = pdf/pdf_eval
        
    if policy_known:
        w = logging_policy/eval_policy
#     print(pdf/pdf_eval)
#     print(logging_policy/eval_policy)
    if use_xshift:
        w = r*w
    if dm:
        w = torch.tensor([1])
    
    weight_st = my_bound(w)
    if len(xy.shape) == 2:
        output = model_robust_list[int(logging_action)](torch.tensor(xy))
    else:
        output = model_robust_list[int(logging_action)](torch.tensor(xy).unsqueeze(0))

    Myy = Myy_robust_list[int(logging_action)]
    Myx = Myx_robust_list[int(logging_action)]

    meanY, varY = ru.predict_regression(torch.tensor(weight_st), Myy, Myx, output, mean0, var0)
    return meanY.detach().numpy()
        
        
        

In [159]:
def evaluate_robust_all(logging_index, logging_action, logging_reward,\
                                            eval_index, eval_action, eval_reward,\
                                            eval_logging_index, eval_logging_action, eval_logging_reward,\
                                            xy,label,  pdf, pdf_eval, logging_policy, eval_policy,\
                                            GT, model_robust_list, Myy_robust_list, Myx_robust_list,\
                                            dm = False,use_xshift = True):
    
    reg_loss = []
    triple_loss = []
    triple_r_loss = []
    sn = np.mean(eval_policy[eval_logging_index,eval_logging_action]/logging_policy[eval_logging_index,eval_logging_action])
    snr = np.mean(pdf_eval[eval_logging_index]/pdf[eval_logging_index])
    tmp_pred = np.zeros((len(eval_index),n_class))
    for i in range(n_class):
        tmp_eval = predict_regression(model_robust_list, Myy_robust_list, Myx_robust_list,\
                                        i,xy[eval_index],  pdf[eval_index], pdf_eval[eval_index], \
                                        logging_policy[eval_index,i], eval_policy[eval_index,i],\
                                        dm = dm,use_xshift = use_xshift)
        tmp_eval = tmp_eval.reshape(-1,len(eval_index))
        tmp_pred[:,i] = tmp_eval
        
    eval_logging_pred = np.zeros((len(eval_logging_index),n_class))
    for i in range(n_class):
        tmp_eval = predict_regression(model_robust_list, Myy_robust_list, Myx_robust_list,\
                                        i,xy[eval_logging_index],  pdf[eval_logging_index], pdf_eval[eval_logging_index], \
                                        logging_policy[eval_logging_index,i], eval_policy[eval_logging_index,i],\
                                        dm = dm,use_xshift = use_xshift)
        tmp_eval = tmp_eval.reshape(-1,len(eval_logging_index))
        eval_logging_pred[:,i] = tmp_eval
    
    regression_pred = np.sum(tmp_pred * eval_policy[eval_index,:],axis = 1)
    
    
    predict_logging = np.zeros(len(eval_logging_index))
    for i in range(len(eval_logging_index)):
        predict_logging[i] =predict_regression(model_robust_list, Myy_robust_list, Myx_robust_list,\
                                        eval_logging_action[i],xy[eval_logging_index[i]],  pdf[eval_logging_index[i]], pdf_eval[eval_logging_index[i]], \
                                        logging_policy[eval_logging_index[i],eval_logging_action[i]], eval_policy[eval_logging_index[i],eval_logging_action[i]],\
                                        dm = dm,use_xshift = use_xshift)
    
    w = eval_policy[eval_logging_index,eval_logging_action]/logging_policy[eval_logging_index,eval_logging_action]
    r = pdf_eval[eval_logging_index]/pdf[eval_logging_index]
#     no r
    triple = w* (eval_logging_reward - eval_logging_pred[np.arange(len(eval_logging_index)),eval_logging_action]) +\
                    regression_pred
    
    
    sn_triple = w/sn *(eval_logging_reward - eval_logging_pred[np.arange(len(eval_index)),eval_logging_action]) + \
                    regression_pred
    
#     r
    triple_r = w*r* (eval_logging_reward - eval_logging_pred[np.arange(len(eval_logging_index)),eval_logging_action]) +\
                    regression_pred
    
    
    sn_triple_r = w/sn*r/snr *(eval_logging_reward - eval_logging_pred[np.arange(len(eval_index)),eval_logging_action]) + \
                    regression_pred
  


    return (np.mean(regression_pred)-GT)**2,(np.mean(triple)-GT)**2,(np.mean(sn_triple)-GT)**2,(np.mean(triple_r)-GT)**2,(np.mean(sn_triple_r)-GT)**2



In [160]:
model_list = [
    'known ips',
    'known snips',
    'known ips-r',
#     'unknown ips',
#     'unknown snips',
#     'unknown ips-r',
    'known dm',
    'known dmdr',
    'known sndmdr',
    'known dmdr-r',
    'known sndmdr-r',
    
#     'known robust without xshift',
#     'known triple without xshift',
#     'known triple-r without xshift',
    'known robust',
    'known triple',
    'known sntriple',
    'known triple-r',
    'known sntriple-r'
    
#     'unknown dm',
#     'unknown dmdr',
#     'unknown dmdr-r',
#     'unknown robust without xshift',
#     'unknown triple without xshift',
#     'unknown triple-r without xshift',
#     'unknown robust',
#     'unknown triple',
#     'unknown triple-r'
]
len(model_list)

13

In [161]:
def estimate_GT_with_eval_policy(xy,label,eval_policy):
    eval_policy = eval_policy[train_size:]
    
    xy = xy[train_size:]
    
    label = label[train_size:]
    result = 0
    

    for i in range(len(xy)):
        result += eval_policy[i,label[i]]
    return result/len(eval_policy)
    
    

In [162]:
def return_dict_with_list(size_list):
    d = {}
    for i in range(len(size_list)):
        d[size_list[i]] = []
    return d

In [163]:
# ips
from tqdm import tqdm

# logging_a_list = [0.5,0.8,1.2]
# logging_b_list = [0.1,0.2,0.4]
size_list = [100,200,500,1000,2000]
train_size_list = [1000,2000,3000,len(train_xy)]
# train_size_list = [1000]
logging_policy_list = [[0.7,0.2],[0.5,0.2],[0.1,0.1]]
logging_xshift_list = [[0.9,0.2],[0.8,0.2],[0.7,0.2],[0.9,0.3],[0.8,0.3],[0.7,0.3]]


cc = 0    
frame = {}
frame['model'] = model_list

for bandit_train_size in train_size_list:
    for logging_interval in logging_policy_list:
        for logging_a, logging_b in logging_xshift_list:
            cc += 1
            print(cc)
            logging_policy, eval_policy = generate_policy_soften(logging_interval[0],logging_interval[1])

            GT = estimate_GT_with_eval_policy(xy,label,eval_policy)


            ips = return_dict_with_list(size_list)
            snips = return_dict_with_list(size_list)
            ips_r =return_dict_with_list(size_list)

            robust = return_dict_with_list(size_list)
            triple = return_dict_with_list(size_list)
            triple_r = return_dict_with_list(size_list)
            sntriple = return_dict_with_list(size_list)
            sntriple_r = return_dict_with_list(size_list)

            dm = return_dict_with_list(size_list)
            dmdr = return_dict_with_list(size_list)
            sndmdr = return_dict_with_list(size_list)
            dmdr_r = return_dict_with_list(size_list)
            sndmdr_r = return_dict_with_list(size_list)



            pdf, mean_logging, std_logging = get_distribution(fc,logging_a, logging_b)
            pdf_eval, mean_eval, std_eval = get_distribution(fc,eval_a, eval_b)


            for _ in tqdm(range(10)):

                logging_index, logging_action, logging_reward = \
                        sample_logging_data(train_xy, train_label, bandit_train_size, pdf, pdf_eval, logging_policy, eval_policy)

                eval_index, eval_action, eval_reward = \
                            sample_eval_data(test_xy, test_label, 100, pdf, pdf_eval, logging_policy, eval_policy)  

                model_robust_list, Myy_robust_list, Myx_robust_list = train_robust_regression(logging_index, logging_action, logging_reward,\
                                                eval_index, eval_action, eval_reward,\
                                                xy,label,  pdf, pdf_eval, logging_policy, eval_policy,\
                                                dm=False, use_xshift = True)
                model_dm_robust_list, Myy_dm_robust_list, Myx_dm_robust_list = train_robust_regression(logging_index, logging_action, logging_reward,\
                                                eval_index, eval_action, eval_reward,\
                                                xy,label,  pdf, pdf_eval, logging_policy, eval_policy,\
                                                dm=True, use_xshift = True)


                for size in size_list:

                    eval_index, eval_action, eval_reward = \
                            sample_eval_data(test_xy, test_label, size, pdf, pdf_eval, logging_policy, eval_policy)

                    eval_logging_index, eval_logging_action, eval_logging_reward = \
                            sample_eval_logging_data(test_xy, test_label, size, pdf, pdf_eval, logging_policy, eval_policy)

                    res = evaluate_ips(logging_index, logging_action, logging_reward,\
                                     eval_logging_index, eval_logging_action, eval_logging_reward,\
                                     xy,label,  pdf, pdf_eval, logging_policy, eval_policy,GT)

                    ips[size].append(res[0])
                    snips[size].append(res[1])
                    ips_r[size].append(res[2])


    #                 ips_unknown[size].append(res[3])
    #                 snips_unknown[size].append(res[4])



                    res = evaluate_robust_all(logging_index, logging_action, logging_reward,\
                                                                eval_index, eval_action, eval_reward,\
                                                                eval_logging_index, eval_logging_action, eval_logging_reward,\
                                                                xy,label,  pdf, pdf_eval, logging_policy, eval_policy,GT,\
                                                                model_robust_list, Myy_robust_list, Myx_robust_list,\
                                                                dm = False,use_xshift = False)
                    robust[size].append(res[0])
                    triple[size].append(res[1])
                    sntriple[size].append(res[2])
                    triple_r[size].append(res[3])
                    sntriple_r[size].append(res[4])






                    res = evaluate_robust_all(logging_index, logging_action, logging_reward,\
                                        eval_index, eval_action, eval_reward,\
                                        eval_logging_index, eval_logging_action, eval_logging_reward,\
                                        xy,label,  pdf, pdf_eval, logging_policy, eval_policy,GT,\
                                        model_dm_robust_list, Myy_dm_robust_list, Myx_dm_robust_list,\
                                        dm = True,use_xshift = False)
                    dm[size].append(res[0])
                    dmdr[size].append(res[1])
                    sndmdr[size].append(res[2])
                    dmdr_r[size].append(res[3])
                    sndmdr_r[size].append(res[4])




                for k in range(len(size_list)):
                    size = size_list[k]
                    policy_name = str(bandit_train_size) +'_'+ str(size_list[k]) + str(logging_interval) + str(logging_a) + '_' + str(logging_b)
                    result_list = []



                    result_list.append(np.sqrt(np.mean(ips[size])))
                    result_list.append(np.sqrt(np.mean(snips[size])))
                    result_list.append(np.sqrt(np.mean(ips_r[size])))

                    result_list.append(np.sqrt(np.mean(dm[size])))
                    result_list.append(np.sqrt(np.mean(dmdr[size])))
                    result_list.append(np.sqrt(np.mean(sndmdr[size])))
                    result_list.append(np.sqrt(np.mean(dmdr_r[size])))
                    result_list.append(np.sqrt(np.mean(sndmdr_r[size])))

                    result_list.append(np.sqrt(np.mean(robust[size])))
                    result_list.append(np.sqrt(np.mean(triple[size])))
                    result_list.append(np.sqrt(np.mean(sntriple[size])))
                    result_list.append(np.sqrt(np.mean(triple_r[size])))
                    result_list.append(np.sqrt(np.mean(sntriple_r[size])))

                    frame[policy_name] = result_list
frame = pd.DataFrame(frame).round(5)
print(frame)
# frame.to_csv('result2/newadv_noxshift.csv',index = False)


1


100%|███████████████████████████████████████████| 10/10 [00:31<00:00,  3.15s/it]


2


100%|███████████████████████████████████████████| 10/10 [00:31<00:00,  3.11s/it]


3


...

42


100%|███████████████████████████████████████████| 10/10 [01:08<00:00,  6.83s/it]


43


 30%|█████████████▏                              | 3/10 [00:23<00:54,  7.85s/it]


KeyboardInterrupt: 

In [166]:
frame = pd.DataFrame(frame).round(5)
frame

model  1000_100[0.7, 0.2]0.9_0.2  1000_200[0.7, 0.2]0.9_0.2  \
0          known ips                    0.12221                    0.12308   
1        known snips                    0.03089                    0.05136   
2        known ips-r                    0.13793                    0.14144   
3           known dm                    0.09514                    0.09707   
4         known dmdr                    0.05686                    0.05226   
5       known sndmdr                    0.05711                    0.05241   
6       known dmdr-r                    0.04217                    0.04027   
7     known sndmdr-r                    0.04270                    0.04073   
8       known robust                    0.12188                    0.11656   
9       known triple                    0.05100                    0.06832   
10    known sntriple                    0.04196                    0.06318   
11    known triple-r                    0.06776                    0.05917   
12  known sntriple-r                    0.06125                    0.05527   

    1000_500[0.7, 0.2]0.9_0.2  1000_1000[0.7, 0.2]0.9_0.2  \
0                     0.04651                     0.03584   
1                     0.02711                     0.00809   
2                     0.04739                     0.04661   
3                     0.09875                     0.09454   
4                     0.06718                     0.03535   
5                     0.06431                     0.03367   
6                     0.04839                     0.02618   
7                     0.04744                     0.02362   
8                     0.11134                     0.11298   
9                     0.06102                     0.02649   
10                    0.05786                     0.02614   
11                    0.05384                     0.02193   
12                    0.05210                     0.01960   

    1000_2000[0.7, 0.2]0.9_0.2  1000_100[0.7, 0.2]0.8_0.2  \
0                      0.02542                    0.14249   
1                      0.02218                    0.02623   
2                      0.02330                    0.18554   
3                      0.09398                    0.10915   
4                      0.03689                    0.08357   
5                      0.03683                    0.06987   
6                      0.01928                    0.10279   
7                      0.01956                    0.07851   
8                      0.11607                    0.07283   
9                      0.03828                    0.05401   
10                     0.03927                    0.04899   
11                     0.02500                    0.08218   
12                     0.02549                    0.06911   

    1000_200[0.7, 0.2]0.8_0.2  1000_500[0.7, 0.2]0.8_0.2  \
0                     0.05166                    0.04968   
1                     0.01701                    0.02016   
2                     0.10198                    0.02805   
3                     0.10321                    0.10605   
4                     0.04421                    0.03001   
5                     0.04165                    0.02884   
6                     0.05906                    0.01613   
7                     0.05613                    0.01618   
8                     0.08921                    0.07930   
9                     0.04953                    0.04285   
10                    0.04903                    0.04092   
11                    0.07407                    0.02797   
12                    0.07236                    0.02788   

    1000_1000[0.7, 0.2]0.8_0.2  ...  3000_100[0.7, 0.2]0.7_0.3  \
0                      0.03867  ...                    0.05901   
1                      0.02076  ...                    0.02566   
2                      0.03351  ...                    0.18432   
3                      0.11068  ...                    0.06730   
4                      0.04913  ...      

In [ ]:
frame_len = frame.copy()

In [ ]:
frame.to_csv('result2/opt_result1.csv',index = False)

In [ ]:
tmp_res

In [405]:
result_list

[0.025501193274650058,
 0.018239821112019276,
 0.016477342001940082,
 0.01589913621870108,
 0.026133660873687365,
 0.02510542971825441,
 0.02425933609964079]

In [ ]:
a = []
b = []
c = []
k = 0
while k < len(ips):
    a.append(ips[k])
    k += 1
    b.append(ips[k])
    k += 1
    c.append(ips[k])
    k += 1
print(np.sqrt(np.mean(a)),np.sqrt(np.mean(b)),np.sqrt(np.mean(c)))  

In [389]:
a = []
b = []
c = []
k = 0
while k < len(ips):
    a.append(snips[k])
    k += 1
    b.append(snips[k])
    k += 1
    c.append(snips[k])
    k += 1
print(np.sqrt(np.mean(a)),np.sqrt(np.mean(b)),np.sqrt(np.mean(c)))  

0.03102377622022736 0.020515294875127845 0.011759379455412883


In [392]:
a = []
b = []
c = []
k = 0
while k < len(ips):
    a.append(triple_noxshift[k])
    k += 1
    b.append(triple_noxshift[k])
    k += 1
    c.append(triple_noxshift[k])
    k += 1
print(np.sqrt(np.mean(a)),np.sqrt(np.mean(b)),np.sqrt(np.mean(c)))  

0.03073353914866272 0.018495907732385557 0.011262325168644553


In [394]:
a = []
b = []
c = []
k = 0
while k < len(ips):
    a.append(sntriple_noxshift[k])
    k += 1
    b.append(sntriple_noxshift[k])
    k += 1
    c.append(sntriple_noxshift[k])
    k += 1
print(np.sqrt(np.mean(a)),np.sqrt(np.mean(b)),np.sqrt(np.mean(c)))  

0.02971956963320808 0.01825370891149572 0.011315999262818321
